# Sample Workflow for d3rlpy Experiments

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import math
import subprocess
import os
import d3rlpy
plt.style.use('matplotlibrc')

from Python.data_sampler import *

## Building an MDPDataset

We first read in a large batch of samples from the file. As `d3rlpy` wants it in the form (observations, actions, rewards, terminal flags), we go ahead and do that. Here's a helper function to get a dataset from a list of chunks of your choosing.

In [2]:
def get_dataset(chunks : list, batch_size=30000, 
                path="collected_data/rl_det_small.txt") -> d3rlpy.dataset.MDPDataset :
    random.seed(0)
    samples = DataSampler(path_to_data=path)
    states = []
    actions = []
    rewards = []
    next_states = []
    for chunk in chunks:
        samples.use_chunk(chunk)
        samples.read_chunk()
        [statesChunk, actionsChunk, rewardsChunk, nextStatesChunk] = samples.get_batch(batch_size)
        states.append(statesChunk)
        actions.append(actionsChunk)
        rewards.append(rewardsChunk)
        next_states.append(nextStatesChunk)
    states = torch.cat(states)
    actions = torch.cat(actions)
    rewards = torch.cat(rewards)
    next_states = torch.cat(next_states)
    terminals = np.zeros(len(states))
    terminals[::100] = 1 #episode length 100, change if necessary
    print(states.shape)
    dataset = d3rlpy.dataset.MDPDataset(states.numpy(), 
                                        actions.numpy(), 
                                        rewards.numpy(), terminals)
    return dataset

We can build the dataset from there, just like this, and split into train and test sets.

In [3]:
dataset = get_dataset([3,5,7,9], path="collected_data/rl_det_small.txt")

[ 0.00000000e+00  7.95731469e+08 -8.17891077e-02 -1.19999531e-03
  7.39998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  2.09713430e-01 -2.63658359e-01  6.00000000e-01]
Read chunk # 4 out of 10000
[ 0.00000000e+00  7.95731469e+08  1.24610892e-01  2.40000469e-03
 -7.60001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -2.20016566e-01  3.79282423e-01 -6.00000000e-01]
Read chunk # 6 out of 10000
[ 0.00000000e+00  7.95731469e+08 -9.01891077e-02  1.08000047e-02
  3.99986580e-04  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -3.17973088e-02 -2.40776052e-01  6.00000000e-01]
Read chunk # 8 out of 10000
[ 0.00000000e+00  7.95731469e+08  6.91108923e-02 -5.99999531e-03
 -6.00001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -1.42355434e-01  2.22081792e-01 -6.00000000e-01]
Read chunk # 10 out of 10000
torch.Size([111080, 6])


In [4]:
print("The behavior policy value statistics are:")
dataset.compute_stats()['return']

The behavior policy value statistics are:


{'mean': -4.1227446,
 'std': 2.4676569,
 'min': -12.578855,
 'max': 0.0,
 'histogram': (array([ 26,   9,   7,   7,   8,   7,  10,  13,  27,  54,  56,  73, 109,
          84, 186, 148, 124,  83,  67,  13]),
  array([-12.578855 , -11.949912 , -11.320969 , -10.692026 , -10.063084 ,
          -9.434141 ,  -8.805199 ,  -8.176255 ,  -7.5473127,  -6.9183702,
          -6.2894273,  -5.6604843,  -5.031542 ,  -4.4025993,  -3.7736564,
          -3.1447136,  -2.515771 ,  -1.8868282,  -1.2578855,  -0.6289427,
           0.       ], dtype=float32))}

In [5]:
from sklearn.model_selection import train_test_split
train_episodes, test_episodes = train_test_split(dataset, test_size=0.2)

## Setting up an Algorithm

In [6]:
from d3rlpy.dynamics import ProbabilisticEnsembleDynamics
from d3rlpy.metrics.scorer import dynamics_observation_prediction_error_scorer
from d3rlpy.metrics.scorer import dynamics_reward_prediction_error_scorer
from d3rlpy.metrics.scorer import dynamics_prediction_variance_scorer

dynamics = d3rlpy.dynamics.ProbabilisticEnsembleDynamics(learning_rate=1e-4, use_gpu=False)

# same as algorithms
dynamics.fit(train_episodes,
             eval_episodes=test_episodes,
             n_epochs=15, tensorboard_dir='runs',
             scorers={
                'observation_error': dynamics_observation_prediction_error_scorer,
                'reward_error': dynamics_reward_prediction_error_scorer,
                'variance': dynamics_prediction_variance_scorer,
             })


#from d3rlpy.algos import MOPO

# load trained dynamics model
#dynamics = ProbabilisticEnsembleDynamics.from_json('<path-to-params.json>/params.json')
#dynamics.load_model('<path-to-model>/model_xx.pt')

# give mopo as generator argument.
#mopo = MOPO(dynamics=dynamics)


2022-04-07 19:44.08 [debug    ] RoundIterator is selected.
2022-04-07 19:44.08 [info     ] Directory is created at d3rlpy_logs/ProbabilisticEnsembleDynamics_20220407194408
2022-04-07 19:44.08 [debug    ] Building models...
2022-04-07 19:44.08 [debug    ] Models have been built.
2022-04-07 19:44.08 [info     ] Parameters are saved to d3rlpy_logs/ProbabilisticEnsembleDynamics_20220407194408/params.json params={'action_scaler': None, 'batch_size': 100, 'discrete_action': False, 'encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'gamma': 1.0, 'generated_maxlen': 100000, 'learning_rate': 0.0001, 'n_ensembles': 5, 'n_frames': 1, 'n_steps': 1, 'optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0.0001, 'amsgrad': False}, 'real_ratio': 1.0, 'reward_scaler': None, 'scaler': None, 'use_gpu': None, 'variance_type': 'max', 'algorithm': 'ProbabilisticEnsembleDynamics', 'observation_shape': 

Epoch 1/15:   0%|          | 0/878 [00:00<?, ?it/s]

2022-04-07 19:44.15 [info     ] ProbabilisticEnsembleDynamics_20220407194408: epoch=1 step=878 epoch=1 metrics={'time_sample_batch': 0.00017330478155531482, 'time_algorithm_update': 0.007003860756042064, 'loss': -140.08404518904882, 'time_step': 0.007243874947411052, 'observation_error': 0.003597034412639334, 'reward_error': 0.00011350073554916066, 'variance': 3.1100626179800764e-08} step=878
2022-04-07 19:44.15 [info     ] Model parameters are saved to d3rlpy_logs/ProbabilisticEnsembleDynamics_20220407194408/model_878.pt


Epoch 2/15:   0%|          | 0/878 [00:00<?, ?it/s]

2022-04-07 19:44.23 [info     ] ProbabilisticEnsembleDynamics_20220407194408: epoch=2 step=1756 epoch=2 metrics={'time_sample_batch': 0.00016093851493540005, 'time_algorithm_update': 0.0073113324701921814, 'loss': -164.6129103032766, 'time_step': 0.007528732197702882, 'observation_error': 0.0036057853225010387, 'reward_error': 8.001452038873771e-05, 'variance': 4.161028056893669e-08} step=1756
2022-04-07 19:44.23 [info     ] Model parameters are saved to d3rlpy_logs/ProbabilisticEnsembleDynamics_20220407194408/model_1756.pt


...

Epoch 14/15:   0%|          | 0/878 [00:00<?, ?it/s]

2022-04-07 19:45.59 [info     ] ProbabilisticEnsembleDynamics_20220407194408: epoch=14 step=12292 epoch=14 metrics={'time_sample_batch': 0.00015231417088953944, 'time_algorithm_update': 0.007223932781089139, 'loss': -182.90584715406553, 'time_step': 0.007428983892558104, 'observation_error': 0.0035792091150054933, 'reward_error': 5.76587521316201e-06, 'variance': 3.14386667274566e-08} step=12292
2022-04-07 19:45.59 [info     ] Model parameters are saved to d3rlpy_logs/ProbabilisticEnsembleDynamics_20220407194408/model_12292.pt


Epoch 15/15:   0%|          | 0/878 [00:00<?, ?it/s]

2022-04-07 19:46.07 [info     ] ProbabilisticEnsembleDynamics_20220407194408: epoch=15 step=13170 epoch=15 metrics={'time_sample_batch': 0.00015121087397964233, 'time_algorithm_update': 0.007162831095736771, 'loss': -183.1120646048787, 'time_step': 0.007364273342836419, 'observation_error': 0.0035801331781499185, 'reward_error': 2.8068526554637735e-06, 'variance': 7.271423195302175e-08} step=13170
2022-04-07 19:46.07 [info     ] Model parameters are saved to d3rlpy_logs/ProbabilisticEnsembleDynamics_20220407194408/model_13170.pt


[(1,
  {'time_sample_batch': 0.00017330478155531482,
   'time_algorithm_update': 0.007003860756042064,
   'loss': -140.08404518904882,
   'time_step': 0.007243874947411052,
   'observation_error': 0.003597034412639334,
   'reward_error': 0.00011350073554916066,
   'variance': 3.1100626179800764e-08}),
 (2,
  {'time_sample_batch': 0.00016093851493540005,
   'time_algorithm_update': 0.0073113324701921814,
   'loss': -164.6129103032766,
   'time_step': 0.007528732197702882,
   'observation_error': 0.0036057853225010387,
   'reward_error': 8.001452038873771e-05,
   'variance': 4.161028056893669e-08}),
 (3,
  {'time_sample_batch': 0.00016076146605889183,
   'time_algorithm_update': 0.007144571708383756,
   'loss': -161.71328518982628,
   'time_step': 0.007364843049190582,
   'observation_error': 0.0035862366632322747,
   'reward_error': 5.786977457846211e-05,
   'variance': 2.992409110233798e-08}),
 (4,
  {'time_sample_batch': 0.00016010323526647477,
   'time_algorithm_update': 0.0072394568

In [7]:
%load_ext tensorboard
%tensorboard --logdir runs

Reusing TensorBoard on port 6006 (pid 83036), started 0:02:45 ago. (Use '!kill 83036' to kill it.)

In [9]:
from d3rlpy.algos import COMBO
from d3rlpy.metrics.scorer import td_error_scorer
from d3rlpy.metrics.scorer import average_value_estimation_scorer
from d3rlpy.metrics.scorer import initial_state_value_estimation_scorer

# calculate metrics with test dataset
#ave_error_init = average_value_estimation_scorer(model, test_episodes)
#print(ave_error_init)

from d3rlpy.preprocessing import MinMaxActionScaler
action_scaler = MinMaxActionScaler(minimum=-0.6, maximum=0.6)

model = COMBO(q_func_factory='mean', #qr -> quantile regression q function, but you don't have to use this
            reward_scaler='standard',
              action_scaler = action_scaler,
        dynamics=dynamics,
          actor_learning_rate=3e-5, 
          critic_learning_rate=0.0003, 
            use_gpu=False) #change it to true if you have one
model.build_with_dataset(dataset)

model.fit(train_episodes,
        eval_episodes=test_episodes,
        n_epochs=5, 
        tensorboard_dir='runs',
        scorers={
            'td_error': td_error_scorer,
            'init_value': initial_state_value_estimation_scorer,
            'ave_value': average_value_estimation_scorer
        })

2022-04-07 19:47.23 [debug    ] RoundIterator is selected.
2022-04-07 19:47.23 [info     ] Directory is created at d3rlpy_logs/COMBO_20220407194723
2022-04-07 19:47.23 [debug    ] Fitting action scaler...       action_scaler=min_max
2022-04-07 19:47.23 [debug    ] Fitting reward scaler...       reward_scaler=standard
2022-04-07 19:47.23 [warning  ] Skip building models since they're already built.
2022-04-07 19:47.23 [info     ] Parameters are saved to d3rlpy_logs/COMBO_20220407194723/params.json params={'action_scaler': {'type': 'min_max', 'params': {'minimum': array(-0.6), 'maximum': array(0.6)}}, 'actor_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'actor_learning_rate': 3e-05, 'actor_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'batch_size': 256, 'conservative_weight': 1.0, 'critic_encoder_factory': {'type': 'default', 'params': {'activation'

Epoch 1/5:   0%|          | 0/343 [00:00<?, ?it/s]

2022-04-07 19:47.25 [debug    ] 250000 transitions are generated. fake_transitions=250000 real_transitions=87813
2022-04-07 19:47.35 [info     ] COMBO_20220407194723: epoch=1 step=343 epoch=1 metrics={'time_sample_batch': 0.00032083772709348804, 'time_algorithm_update': 0.026542953430042322, 'critic_loss': 9.42205670971217, 'actor_loss': -0.8466464337776075, 'temp_loss': 4.4922140492642235, 'temp': 0.9834773337180691, 'time_step': 0.026942127300073386, 'td_error': 0.7837354701278182, 'init_value': 1.1931893825531006, 'ave_value': 1.190522290886045} step=343
2022-04-07 19:47.35 [info     ] Model parameters are saved to d3rlpy_logs/COMBO_20220407194723/model_343.pt


Epoch 2/5:   0%|          | 0/343 [00:00<?, ?it/s]

2022-04-07 19:47.45 [info     ] COMBO_20220407194723: epoch=2 step=686 epoch=2 metrics={'time_sample_batch': 0.0005197184426443917, 'time_algorithm_update': 0.027178133889468003, 'critic_loss': 10.605080838106117, 'actor_loss': -2.214353049461765, 'temp_loss': 4.455888712718952, 'temp': 0.952205315449495, 'time_step': 0.027776420637747986, 'td_error': 0.5823717932963601, 'init_value': 2.4637928009033203, 'ave_value': 2.458846289037382} step=686
2022-04-07 19:47.45 [info     ] Model parameters are saved to d3rlpy_logs/COMBO_20220407194723/model_686.pt


...

Epoch 4/5:   0%|          | 0/686 [00:00<?, ?it/s]

2022-04-07 19:48.27 [debug    ] 250000 transitions are generated. fake_transitions=750000 real_transitions=87813
2022-04-07 19:48.29 [info     ] COMBO_20220407194723: epoch=4 step=2058 epoch=4 metrics={'time_sample_batch': 0.0005001239804415244, 'time_algorithm_update': 0.026712157288376167, 'critic_loss': 10.128434152714364, 'actor_loss': -5.458631817175418, 'temp_loss': 4.000172061753343, 'temp': 0.8477194980872963, 'time_step': 0.02729754788534982, 'td_error': 1.3077000815826092, 'init_value': 5.953140735626221, 'ave_value': 5.940911734209147} step=2058
2022-04-07 19:48.29 [info     ] Model parameters are saved to d3rlpy_logs/COMBO_20220407194723/model_2058.pt


Epoch 5/5:   0%|          | 0/686 [00:00<?, ?it/s]

2022-04-07 19:48.50 [info     ] COMBO_20220407194723: epoch=5 step=2744 epoch=5 metrics={'time_sample_batch': 0.0005241562951758027, 'time_algorithm_update': 0.028877137701295904, 'critic_loss': 10.412568345361827, 'actor_loss': -6.568842086430541, 'temp_loss': 3.7428856445123433, 'temp': 0.7949885841535063, 'time_step': 0.02948999231132752, 'td_error': 1.877593398146586, 'init_value': 7.151582717895508, 'ave_value': 7.133662610581608} step=2744
2022-04-07 19:48.50 [info     ] Model parameters are saved to d3rlpy_logs/COMBO_20220407194723/model_2744.pt


[(1,
  {'time_sample_batch': 0.00032083772709348804,
   'time_algorithm_update': 0.026542953430042322,
   'critic_loss': 9.42205670971217,
   'actor_loss': -0.8466464337776075,
   'temp_loss': 4.4922140492642235,
   'temp': 0.9834773337180691,
   'time_step': 0.026942127300073386,
   'td_error': 0.7837354701278182,
   'init_value': 1.1931893825531006,
   'ave_value': 1.190522290886045}),
 (2,
  {'time_sample_batch': 0.0005197184426443917,
   'time_algorithm_update': 0.027178133889468003,
   'critic_loss': 10.605080838106117,
   'actor_loss': -2.214353049461765,
   'temp_loss': 4.455888712718952,
   'temp': 0.952205315449495,
   'time_step': 0.027776420637747986,
   'td_error': 0.5823717932963601,
   'init_value': 2.4637928009033203,
   'ave_value': 2.458846289037382}),
 (3,
  {'time_sample_batch': 0.0005254707253014034,
   'time_algorithm_update': 0.027556525374988077,
   'critic_loss': 10.050928521781898,
   'actor_loss': -3.7198425628701033,
   'temp_loss': 4.285134872264139,
   'tem

## Off-Policy Evaluation

We do get some metrics on a test set of initial state value and average value. However, these estimates (using the critic's Q-function) of model performance are biased. They're useful for validation during training, but not much else. Instead, we fit a Q-function to the data (or a separate dataset, as I've done here) separately and evaluate the model's performance on it.

Feel free to change the chunks and number of steps.

In [12]:
model.save_policy('comboDet.pt')

/Users/hetankevin/miniforge3/lib/python3.9/site-packages/d3rlpy/preprocessing/action_scalers.py:204: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  minimum = torch.tensor(
/Users/hetankevin/miniforge3/lib/python3.9/site-packages/d3rlpy/preprocessing/action_scalers.py:207: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  maximum = torch.tensor(


In [10]:
from d3rlpy.ope import FQE
# metrics to evaluate with
from d3rlpy.metrics.scorer import soft_opc_scorer

ope_dataset = get_dataset([2,4,6,8], path="collected_data/rl_det_small.txt") #change if you'd prefer different chunks
ope_train_episodes, ope_test_episodes = train_test_split(ope_dataset, test_size=0.2)

fqe = FQE(algo=model, action_scaler = action_scaler, use_gpu=False) #change this if you have one!
fqe.fit(ope_train_episodes, eval_episodes=ope_test_episodes, 
        tensorboard_dir='runs',
        n_epochs=50, n_steps_per_epoch=1000, #change if overfitting/underfitting
        scorers={
           'init_value': initial_state_value_estimation_scorer,
            'ave_value': average_value_estimation_scorer,
           'soft_opc': soft_opc_scorer(return_threshold=0)
        })

[ 0.00000000e+00  7.95731469e+08 -1.03891077e-02 -1.41999953e-02
 -2.10001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -1.78778459e-03 -1.34615461e-02  4.84073546e-02]
Read chunk # 3 out of 10000
[ 0.00000000e+00  7.95731469e+08 -7.24891077e-02 -1.35999953e-02
 -4.20001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.23311010e-02 -1.64283998e-01  6.00000000e-01]
Read chunk # 5 out of 10000
[ 0.00000000e+00  7.95731469e+08  7.01089229e-03 -4.19999531e-03
  7.39998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  2.21623335e-01 -2.86362315e-02 -8.00043364e-02]
Read chunk # 7 out of 10000
[ 0.00000000e+00  7.95731469e+08 -1.03989108e-01 -1.37999953e-02
  7.99998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  2.76352555e-01 -3.26280816e-01  6.00000000e-01]
Read chunk # 9 out of 10000
torch.Size([111080, 6])
2022-04-07 19:48.50 [debug    ] RoundIterator is selected.
2022-04-07 19:48.50 [info     ] Directory is created at d3rlpy_logs/FQE_2022040719485

Epoch 1/50:   0%|          | 0/878 [00:00<?, ?it/s]

/Users/hetankevin/miniforge3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/hetankevin/miniforge3/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


2022-04-07 19:48.53 [info     ] FQE_20220407194850: epoch=1 step=878 epoch=1 metrics={'time_sample_batch': 0.00014098385743508307, 'time_algorithm_update': 0.0019090096879928694, 'loss': 0.0006079547214486205, 'time_step': 0.0021188560542321692, 'init_value': -0.28840816020965576, 'ave_value': -0.28866724583605535, 'soft_opc': nan} step=878


2022-04-07 19:48.53 [info     ] Model parameters are saved to d3rlpy_logs/FQE_20220407194850/model_878.pt


...

2022-04-07 19:50.20 [info     ] Model parameters are saved to d3rlpy_logs/FQE_20220407194850/model_43022.pt


Epoch 50/50:   0%|          | 0/878 [00:00<?, ?it/s]

2022-04-07 19:50.22 [info     ] FQE_20220407194850: epoch=50 step=43900 epoch=50 metrics={'time_sample_batch': 0.0001044140318260106, 'time_algorithm_update': 0.0013078282797255114, 'loss': 0.0048197853892544655, 'time_step': 0.0014610472464072677, 'init_value': -0.7038307189941406, 'ave_value': -0.7038266831843507, 'soft_opc': nan} step=43900


2022-04-07 19:50.22 [info     ] Model parameters are saved to d3rlpy_logs/FQE_20220407194850/model_43900.pt


[(1,
  {'time_sample_batch': 0.00014098385743508307,
   'time_algorithm_update': 0.0019090096879928694,
   'loss': 0.0006079547214486205,
   'time_step': 0.0021188560542321692,
   'init_value': -0.28840816020965576,
   'ave_value': -0.28866724583605535,
   'soft_opc': nan}),
 (2,
  {'time_sample_batch': 0.00014134908862972043,
   'time_algorithm_update': 0.0018845446288992985,
   'loss': 0.0012250064993522321,
   'time_step': 0.00208426798123162,
   'init_value': -0.4084046483039856,
   'ave_value': -0.4085957081498684,
   'soft_opc': nan}),
 (3,
  {'time_sample_batch': 0.00015009101268099217,
   'time_algorithm_update': 0.0020265109446705883,
   'loss': 0.0021275242328171145,
   'time_step': 0.00224410102687827,
   'init_value': -0.5219352841377258,
   'ave_value': -0.5222797834360767,
   'soft_opc': nan}),
 (4,
  {'time_sample_batch': 0.0001335703430523362,
   'time_algorithm_update': 0.0016752519477201213,
   'loss': 0.0029731072644337106,
   'time_step': 0.0018675987161101948,
   '

In [11]:
from d3rlpy.ope import FQE
# metrics to evaluate with
from d3rlpy.metrics.scorer import soft_opc_scorer

ope_dataset = get_dataset([2,4,6,8], path="collected_data/rl_stoch_small.txt") #change if you'd prefer different chunks
ope_train_episodes, ope_test_episodes = train_test_split(ope_dataset, test_size=0.2)

fqe = FQE(algo=model, action_scaler = action_scaler, use_gpu=False) #change this if you have one!
fqe.fit(ope_train_episodes, eval_episodes=ope_test_episodes, 
        tensorboard_dir='runs',
        n_epochs=50, n_steps_per_epoch=1000, #change if overfitting/underfitting
        scorers={
           'init_value': initial_state_value_estimation_scorer,
            'ave_value': average_value_estimation_scorer,
           'soft_opc': soft_opc_scorer(return_threshold=0)
        })

[ 0.00000000e+00  7.95731469e+08  1.39310892e-01  1.82000047e-02
 -1.00013420e-04  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -7.50230117e-02  3.69851546e-01 -6.00000000e-01]
Read chunk # 3 out of 10000
[ 0.00000000e+00  7.95731469e+08 -1.15389108e-01  1.64000047e-02
 -8.80001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -3.08831172e-01 -2.48178665e-01  6.00000000e-01]
Read chunk # 5 out of 10000
[ 0.00000000e+00  7.95731469e+08 -1.28589108e-01  1.20000047e-02
  1.99998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  7.75212759e-03 -3.52719043e-01  6.00000000e-01]
Read chunk # 7 out of 10000
[ 0.00000000e+00  7.95731469e+08 -1.03989108e-01  7.00000469e-03
 -8.60001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -2.65974295e-01 -2.19295880e-01  6.00000000e-01]
Read chunk # 9 out of 10000
torch.Size([111080, 6])
2022-04-07 19:50.22 [debug    ] RoundIterator is selected.
2022-04-07 19:50.22 [info     ] Directory is created at d3rlpy_logs/FQE_2022040719502

Epoch 1/50:   0%|          | 0/878 [00:00<?, ?it/s]

2022-04-07 19:50.24 [info     ] FQE_20220407195022: epoch=1 step=878 epoch=1 metrics={'time_sample_batch': 0.00010922530789038586, 'time_algorithm_update': 0.0013269965360810925, 'loss': 0.0009169417545932128, 'time_step': 0.0014889957149914021, 'init_value': -0.3336537480354309, 'ave_value': -0.3335277750286961, 'soft_opc': nan} step=878


2022-04-07 19:50.24 [info     ] Model parameters are saved to d3rlpy_logs/FQE_20220407195022/model_878.pt


Epoch 2/50:   0%|          | 0/878 [00:00<?, ?it/s]

...

2022-04-07 19:51.45 [info     ] Model parameters are saved to d3rlpy_logs/FQE_20220407195022/model_43022.pt


Epoch 50/50:   0%|          | 0/878 [00:00<?, ?it/s]

2022-04-07 19:51.47 [info     ] FQE_20220407195022: epoch=50 step=43900 epoch=50 metrics={'time_sample_batch': 0.00012361921862209034, 'time_algorithm_update': 0.001523890093410205, 'loss': 0.009692584044443269, 'time_step': 0.0017012363142739125, 'init_value': -0.8151191473007202, 'ave_value': -0.8166476551819799, 'soft_opc': nan} step=43900


2022-04-07 19:51.47 [info     ] Model parameters are saved to d3rlpy_logs/FQE_20220407195022/model_43900.pt


[(1,
  {'time_sample_batch': 0.00010922530789038586,
   'time_algorithm_update': 0.0013269965360810925,
   'loss': 0.0009169417545932128,
   'time_step': 0.0014889957149914021,
   'init_value': -0.3336537480354309,
   'ave_value': -0.3335277750286961,
   'soft_opc': nan}),
 (2,
  {'time_sample_batch': 0.00011259765451209691,
   'time_algorithm_update': 0.0013444882590569777,
   'loss': 0.001964072655541641,
   'time_step': 0.0015072838863642178,
   'init_value': -0.48161980509757996,
   'ave_value': -0.4819707575342267,
   'soft_opc': nan}),
 (3,
  {'time_sample_batch': 0.00012065609386678708,
   'time_algorithm_update': 0.0014837013561796219,
   'loss': 0.0035081159272886825,
   'time_step': 0.0016611716741852989,
   'init_value': -0.5360317230224609,
   'ave_value': -0.5367158918235728,
   'soft_opc': nan}),
 (4,
  {'time_sample_batch': 0.00012901187757696268,
   'time_algorithm_update': 0.0016377564171723732,
   'loss': 0.004300679627766237,
   'time_step': 0.001826680602679763,
   